In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
def MIP_IM():
    budget = 25
    m = 1000
    for index in range(24):
        with open('../MIP/data/networks_prob/graph_spa_500_'+str(index)+'.pickle', "rb") as f:
            main_graph = pickle.load(f)
    
            samples = []
            for j in range(m):
                G = nx.DiGraph()
                for u in main_graph.nodes():
                    G.add_node(u)
                for u,v in main_graph.edges():
                    if main_graph[u][v]['p']> random.random():
                        G.add_edge(u, v)
                samples.append(G)
                
                
            model = Model('influence_maximization_'+str(index))
            mvars = []
            #active nodes
            avars = []
            #seed nodes
            svars = []
            var_seed_dict = {}
            var_active_dict = {}

            for j in range(len(main_graph.nodes())):
                s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                svars.append(s)
                var_seed_dict[j] = s

            for sample_index, sample in enumerate(samples):
                for j in range(len(main_graph.nodes())):
                    a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                    avars.append(a)
                    var_active_dict[(sample_index,j)] = a    

            mvars.append(avars)
            mvars.append(svars)

            obj_expr = quicksum(avars)
            model.setObjective(obj_expr, GRB.MAXIMIZE)
            model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)
            
            for sample_index, sample in enumerate(samples):
                for i in range(len(main_graph.nodes())):
                    neighbors = nx.ancestors(sample, i) 
                    e = len(neighbors)
                    ai = var_active_dict[(sample_index,i)]
                    si = var_seed_dict[i]
                    neighbors_active_vars = []
                    neighbors_seed_vars = []
                    neighbors_active_vars.append(((e+1), ai))
                    neighbors_seed_vars.append(si)
                    for neighbor in neighbors:
                        neighbors_active_vars.append(((e+1), var_active_dict[(sample_index,neighbor)]))
                        neighbors_seed_vars.append(var_seed_dict[neighbor])
                    seed_neighbors = quicksum(neighbors_seed_vars)
                    model.addConstr(ai, GRB.LESS_EQUAL, seed_neighbors)
                    model.addConstr(seed_neighbors, GRB.LESS_EQUAL, LinExpr(neighbors_active_vars))
            try:
                model.optimize()
            except e:
                print(e)
                
            with open('../Git/influence_maximization/experiments/im500/results/mip/base1000/output_'+str(index)+'.txt', "w") as of:    
                for key,value in var_seed_dict.items():
                    if(value.x > 0):
                        print(key, file = of)

In [ ]:
MIP_IM()

Optimize a model with 1000001 rows, 500500 columns and 3100753 nonzeros
Variable types: 0 continuous, 500500 integer (500500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 305408 rows and 0 columns (presolve time = 5s) ...
Presolve removed 305408 rows and 0 columns (presolve time = 10s) ...
Presolve removed 305408 rows and 0 columns (presolve time = 15s) ...
Presolve removed 594604 rows and 289196 columns (presolve time = 20s) ...
Presolve removed 594604 rows and 289196 columns (presolve time = 25s) ...
Presolve removed 594604 rows and 289196 columns
Presolve time: 27.14s
Presolved: 405397 rows, 211304 columns, 1639194 nonzeros
Variable types: 0 continuous, 211304 integer (211304 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 405397 rows, 211

  222545    5.9569563e+04   0.000000e+00   3.436083e+05     82s
  223211    5.9569884e+04   0.000000e+00   3.044503e+05     86s
  223877    5.9570079e+04   0.000000e+00   1.891631e+06     91s
  224543    5.9570436e+04   0.000000e+00   3.106210e+05     95s
  225209    5.9570821e+04   0.000000e+00   2.229226e+05    100s
  226541    6.1719201e+04   0.000000e+00   4.466990e+05    105s
  227207    6.1720100e+04   0.000000e+00   2.932971e+05    110s
  229205    6.4648474e+04   0.000000e+00   2.192142e+05    115s
  230204    6.4818425e+04   0.000000e+00   1.698638e+05    121s
  232902    6.6066768e+04   0.000000e+00   1.427520e+05    125s
  234900    6.6068724e+04   0.000000e+00   1.262133e+05    130s
  237231    6.7067235e+04   0.000000e+00   1.128178e+05    135s
  238896    6.8264478e+04   0.000000e+00   1.396090e+05    141s
  239895    6.8264860e+04   0.000000e+00   1.471576e+05    146s
  240561    6.8264930e+04   0.000000e+00   1.555252e+05    150s
  241893    6.8270439e+04   0.000000e+00